In [1]:
!pip install tensorflow
!pip install numpy
!pip install pandas
!pip install matplotlib

Defaulting to user installation because normal site-packages is not writeable
     ------------------------------------ 376.9/376.9 MB 797.3 kB/s eta 0:00:00
     ---------------------------------------- 3.8/3.8 MB 3.9 MB/s eta 0:00:00
     ---------------------------------------- 3.0/3.0 MB 3.6 MB/s eta 0:00:00
     ------------------------------------ 133.7/133.7 KB 376.4 kB/s eta 0:00:00
     -------------------------------------- 57.5/57.5 KB 502.5 kB/s eta 0:00:00
     ---------------------------------------- 26.4/26.4 MB 2.7 MB/s eta 0:00:00
     ---------------------------------------- 1.1/1.1 MB 1.8 MB/s eta 0:00:00
     ------------------------------------ 413.4/413.4 KB 888.0 kB/s eta 0:00:00
     -------------------------------------- 65.5/65.5 KB 207.7 kB/s eta 0:00:00
     ---------------------------------------- 5.5/5.5 MB 2.5 MB/s eta 0:00:00
     ---------------------------------------- 1.5/1.5 MB 3.3 MB/s eta 0:00:00
     ------------------------------------ 127.8/127.

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
mysql-connector-python 8.0.31 requires protobuf<=3.20.1,>=3.11.0, but you have protobuf 4.25.3 which is incompatible.
You should consider upgrading via the 'C:\Program Files\Python310\python.exe -m pip install --upgrade pip' command.


Defaulting to user installation because normal site-packages is not writeable

You should consider upgrading via the 'C:\Program Files\Python310\python.exe -m pip install --upgrade pip' command.



Defaulting to user installation because normal site-packages is not writeable


You should consider upgrading via the 'C:\Program Files\Python310\python.exe -m pip install --upgrade pip' command.


Defaulting to user installation because normal site-packages is not writeable


You should consider upgrading via the 'C:\Program Files\Python310\python.exe -m pip install --upgrade pip' command.


In [2]:
from pathlib import Path
import imghdr
import os

data_dir = "hair_types"
image_extensions = [".png", ".jpg"]  # add there all your images file extensions

img_type_accepted_by_tf = ["bmp", "gif", "jpeg", "png"]
for filepath in Path(data_dir).rglob("*"):
    if filepath.suffix.lower() in image_extensions:
        if os.path.exists(filepath):  # Check if the file exists
            img_type = imghdr.what(filepath)
            if img_type is None:
                print(f"{filepath} is not an image")
                os.remove(filepath)
            elif img_type not in img_type_accepted_by_tf:
                print(f"{filepath} is a {img_type}, not accepted by TensorFlow")
                os.remove(filepath)
        else:
            print(f"{filepath} does not exist")

hair_types\Curly_Hair\090617-header-mobile.jpg is a webp, not accepted by TensorFlow
hair_types\Curly_Hair\beautiful-young-woman-long-curly-260nw-1212622882.jpg is a webp, not accepted by TensorFlow
hair_types\Curly_Hair\blonde-girl-voluminous-curly-hair-600w-1432618526.jpg is a webp, not accepted by TensorFlow
hair_types\Curly_Hair\GettyImages-646055712_trans_NvBQzQNjv4BqDo1PhbPPiB1HBZscUHLBE78-ICSbD7iLWCuXgZfDuIk.jpg is a webp, not accepted by TensorFlow
hair_types\Curly_Hair\indian-baby-girl-curly-hair-600w-1664313094.jpg is a webp, not accepted by TensorFlow
hair_types\Curly_Hair\photo.jpg is a webp, not accepted by TensorFlow
hair_types\Curly_Hair\Untitled-8.jpg is a webp, not accepted by TensorFlow
hair_types\Straight_Hair\10100463715_c9aae924f1_z.jpg is not an image
hair_types\Straight_Hair\14280152025_6735cb2c8e_z.jpg is not an image
hair_types\Straight_Hair\content_thin-hair_1.jpg is not an image


FileNotFoundError: [Errno 2] No such file or directory: 'hair_types\\Straight_Hair\\graphicstock-young-handsome-asiatic-long-brown-straight-hair-woman-taking-selfie-outdoor-in-the-city-with-a-smartphone-smiling-vanity-technology-social-network-concept_B6fyU2Ok-_SB_PM.jpg'

In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

image_size = (64, 64)
batch_size = 32

train_ds = tf.keras.preprocessing.image_dataset_from_directory(
    "hair_types/",
    validation_split=0.2,
    subset="training",
    seed=1337,
    image_size=image_size,
    batch_size=batch_size,
    labels='inferred',
    label_mode='categorical'
)

val_ds = tf.keras.preprocessing.image_dataset_from_directory(
    "hair_types/",
    validation_split=0.2,
    subset="validation",
    seed=1337,
    image_size=image_size,
    batch_size=batch_size, 
    labels='inferred',
    label_mode='categorical'
)

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

plt.figure(figsize=(10, 10))
for images, labels in train_ds.take(1):
    for i in range(9):
        ax = plt.subplot(3, 3, i + 1)
        plt.imshow(images[i].numpy().astype("uint8"))
        plt.title(int(np.argmax(labels[i])))
        plt.axis("off")
plt.show()

In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from keras.models import Sequential
from keras.layers import Dense

model = Sequential()
model.add(keras.Input(shape=image_size + (3,))) # 64, 64, 3
model.add(layers.Rescaling(1.0 / 255))

model.add(layers.Conv2D(filters=4, kernel_size=16, strides=1, padding='valid', dilation_rate=1))
model.add(layers.Activation("relu"))

model.add(layers.Conv2D(filters=8, kernel_size=8, strides=1, padding='valid', dilation_rate=1))
model.add(layers.Activation("relu"))

model.add(layers.Conv2D(filters=16, kernel_size=4, strides=1, padding='valid', dilation_rate=1))
model.add(layers.Activation("relu"))

model.add(layers.GlobalAveragePooling2D())
model.add(layers.Dense(64))
model.add(layers.Activation("relu"))
model.add(layers.Dense(3))
model.add(layers.Activation("softmax"))

tf.keras.utils.plot_model(model, to_file='model_test.png', show_shapes=True)

epochs = 50

model.compile(
    optimizer=keras.optimizers.Adam(1e-3),
    loss="categorical_crossentropy",
    metrics=["accuracy"],
)
model.fit(train_ds, epochs=epochs, validation_data=val_ds)

In [ ]:
img = keras.preprocessing.image.load_img(
    "hair_types/Curly_Hair/02dac897d1dec9ba8c057a11d041ada8--layered-natural-hair-natural-black-hairstyles.jpg", target_size=image_size
)
img_array = keras.preprocessing.image.img_to_array(img)
img_array = tf.expand_dims(img_array, 0)  # Create batch axis

predictions = model.predict(img_array)
print(
    "This image is %.2f percent curly hair, %.2f percent straight hair, and %.2f percent wavy hair."
    % tuple(predictions[0])
)


NameError: name 'keras' is not defined